# Постановка задачи и загрузка данных

In [ ]:
%%capture
!pip install pymorphy2
import pandas as pd
from ast import literal_eval
import numpy as np
from google.colab import drive
import re
from pymorphy2 import MorphAnalyzer
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

morph = MorphAnalyzer()
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
stop_words = stopwords.words('russian') + stopwords.words('english')

drive.mount('/content/drive')

Эти данные можно получить через API Конструктора ОП.

In [ ]:
!gdown 1oz2LeGhHwgW5Mc4uGp1Xm_NVgvQjvOpv

Failed to retrieve file url:

	Too many users have viewed or downloaded this file recently. Please
	try accessing the file again later. If the file you are trying to
	access is particularly large or is shared with many people, it may
	take up to 24 hours to be able to view or download the file. If you
	still can't access a file after 24 hours, contact your domain
	administrator.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1oz2LeGhHwgW5Mc4uGp1Xm_NVgvQjvOpv

but Gdown can't. Please check connections and permissions.


In [ ]:
df = pd.read_csv('disciplines_entities20230506.csv', names=['id', 'title', 'qualification', 'raw_text', 'author_entities'], header=0)
df['author_entities'] = df.author_entities.apply(literal_eval)
df.head()

,id,title,qualification,raw_text,author_entities
0,2626,Философия,bachelor,Философия. Курс «Философия» даёт целостное пре...,"[Ценности современного гуманизма, Системное мы..."
1,2627,Введение в цифровую культуру и программирование,bachelor,Введение в цифровую культуру и программировани...,"[Цифровое образование, Цифровая этика, Техноло..."
2,2628,Проектирование баз данных,bachelor,Проектирование баз данных. Дисциплина направле...,"[Язык структурированных запросов (SQL), Хранен..."
3,2631,Современные вопросы информатики,bachelor,Современные вопросы информатики. Дисциплина на...,"[Теория информации, Сверточные коды, Линейные ..."
4,2635,Иностранный язык,bachelor,Иностранный язык. Целью освоения дисциплины яв...,[Типология языков]


Что есть в данных?
1. `id` – айди дисциплины
2. `title` – название дисциплины
3. `qualification` – квалификация (бакалавриат/магистратура)
4. `raw_text` – текстовая строка, в которую последовательно сложено название дисциплины, описание, заголовки разделов и заголовки тем. Примерно такой столбец получался в рамках предобработки данных в упражнении 3
5. `author_entities` – учебные сущности, которые разработчики дисциплины привязали к ней в качестве результатов обучения

В задании 3.2 вы предобработали данные. Они вам также понадобятся. Прочитайте этот датасет. Также вам понадобится датасет с учебными сущностями из задания 2.

# Задание 4.1

Реализуйте функцию `search_course`, которая на вход будет принимать строку `prompt` и число `n`. Где `prompt` - это строка (запрос пользователя), а `n` - это целое неотрицательное число, означающее количество дисциплин, которые нужно вывести по этому запросу. Выведенные дисциплины должны быть отранжированы по релевантности запросу.

## Объявление значений prompt и n

In [ ]:
prompt = 'физика'
n = 5

In [ ]:
def preprocess(text):
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(' +', ' ', text)
    words = word_tokenize(text.lower())
    words = [w for w in words if not w in stop_words]
    return [morph.parse(w)[0].normal_form for w in words]

def relevance_score(text, prompt):
    text_lemmas = preprocess(text)
    prompt_lemmas = preprocess(prompt)

    match_count = sum(token in prompt_lemmas for token in text_lemmas)

    return match_count

def search_course(prompt, n, input_df, input_column_name):
    input_df['relevance'] = input_df[input_column_name].apply(lambda x: relevance_score(x, prompt))
    df_sorted = input_df.sort_values('relevance', ascending=False)
    top_courses = df_sorted.head(n)
    return top_courses

In [ ]:
search_course(prompt, n, df, 'raw_text')

,id,title,qualification,raw_text,author_entities,relevance
3822,15563,Установка класса Mega-Science в России и за ру...,master,Установка класса Mega-Science в России и за ру...,"[Принцип работы лазера, Линейные ускорители, К...",17
3864,15649,Установки класса Mega-Science в России и за ру...,master,Установки класса Mega-Science в России и за ру...,"[Принцип работы лазера, Линейные ускорители, К...",17
4017,15921,Подготовительный курс по физике,All_levels,Подготовительный курс по физике. Зачем на вело...,"[Электростатика, Электромагнитные волны, Элект...",16
3499,14946,Подготовительный курс по физике,bachelor,Подготовительный курс по физике. Зачем на вело...,"[Электростатика, Электромагнитные волны, Элект...",16
7631,23169,Подготовительный курс по физике,specialist,Подготовительный курс по физике. Что такое чер...,"[Электростатика, Электромагнитная индукция, Фи...",13


# Задание 4.2

Протестируйте функцию `search_course` на датасете из задания 3.2, используя булев поиск по столбцу `norm`, предварительно обработав `prompt`

In [ ]:
df_task_32 = pd.read_csv('drive/My Drive/data_analysis_data/lab3/preprocessed_df2.csv')
df_task_32.head(5)

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding
0,2626,Философия,bachelor,8.0,6905,Программирование и интернет-технологии,2018,Модуль «Философия+Мышление»,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
1,2626,Философия,bachelor,8.0,6938,Программирование и интернет-технологии,2019,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
2,2626,Философия,bachelor,8.0,6974,Информатика и программирование,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
3,2626,Философия,bachelor,8.0,7014,Программирование и интернет-технологии,2020,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...
4,2626,Философия,bachelor,8.0,7136,Информатика и программирование,2021,Модуль обязательных дисциплин,Курс «Философия» даёт целостное представление ...,Курс Философия даёт целостное представление о ...,Курс Философия даёт целостное представление о ...,"['курс', 'философия', 'давать', 'целостный', '...",[ 3.65205932e-02 7.71842406e-04 6.57333977e-...


## Получение датафрейма из задания 3.2 ранжированного по релевантности запросу `prompt` в количестве n

In [ ]:
df_task_32_with_relevance = search_course(prompt, n, df_task_32, 'norm')
df_task_32_with_relevance

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding,relevance
24775,14946,Подготовительный курс по физике,bachelor,3.0,7003,Программирование и искусственный интеллект,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
24826,14946,Подготовительный курс по физике,bachelor,3.0,7063,Управление интеллектуальной собственностью,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
24838,14946,Подготовительный курс по физике,bachelor,3.0,7085,Цифровые системы управления,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
24837,14946,Подготовительный курс по физике,bachelor,3.0,7084,Компьютерные системы и технологии,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
24836,14946,Подготовительный курс по физике,bachelor,3.0,7083,Материалы фотоники,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12


In [ ]:
writer = open('drive/My Drive/data_analysis_data/lab4/df_task_32_with_relevance.csv', 'w', encoding='utf-8')
writer.write('')
writer.close()
df_task_32_with_relevance.to_csv('drive/My Drive/data_analysis_data/lab4/df_task_32_with_relevance.csv', index=False)

In [ ]:
df_task_32_with_relevance = pd.read_csv('drive/My Drive/data_analysis_data/lab4/df_task_32_with_relevance.csv')
df_task_32_with_relevance

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding,relevance
0,14946,Подготовительный курс по физике,bachelor,3.0,7003,Программирование и искусственный интеллект,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
1,14946,Подготовительный курс по физике,bachelor,3.0,7063,Управление интеллектуальной собственностью,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
2,14946,Подготовительный курс по физике,bachelor,3.0,7085,Цифровые системы управления,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
3,14946,Подготовительный курс по физике,bachelor,3.0,7084,Компьютерные системы и технологии,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
4,14946,Подготовительный курс по физике,bachelor,3.0,7083,Материалы фотоники,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12


## Булев поиск по датафрейму из задания 3.2

In [ ]:
matching_courses = df_task_32_with_relevance[df_task_32_with_relevance['norm'].apply(lambda x: all(word in x for word in preprocess(prompt)))]
matching_courses

,course_id,course_title,qualification,unit_id,up_id,up_title,start_year,module_name,text,punctuation,extra_spaces,norm,embedding,relevance
0,14946,Подготовительный курс по физике,bachelor,3.0,7003,Программирование и искусственный интеллект,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
1,14946,Подготовительный курс по физике,bachelor,3.0,7063,Управление интеллектуальной собственностью,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
2,14946,Подготовительный курс по физике,bachelor,3.0,7085,Цифровые системы управления,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
3,14946,Подготовительный курс по физике,bachelor,3.0,7084,Компьютерные системы и технологии,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12
4,14946,Подготовительный курс по физике,bachelor,3.0,7083,Материалы фотоники,2020,Факультативные дисциплины,Зачем на велосипеде различные скорости? Почему...,Зачем на велосипеде различные скорости Почему ...,Зачем на велосипеде различные скорости Почему ...,"['велосипед', 'различный', 'скорость', 'почему...",[ 5.56915305e-02 4.09553628e-02 6.03928252e-...,12


# Задание 4.3

Протестируйте функцию `search_course` на датасете, который был загружен выше, используя **tf-idf**.

In [ ]:
def search_course_43(prompt, n):
    df['preprocessed_text'] = df['raw_text'].apply(preprocess)
    df['preprocessed_text'] = df['preprocessed_text'].apply(lambda x: ' '.join(x))

    # TF-IDF
    vectorizer = TfidfVectorizer()
    tfidf_matrix = vectorizer.fit_transform(df['preprocessed_text'])

    # Расчет схожести запроса с каждой записью
    prompt_vector = vectorizer.transform(preprocess(prompt))
    similarities = cosine_similarity(tfidf_matrix, prompt_vector)

    # Получение наиболее релевантных записей
    top_indices = similarities.flatten().argsort()[::-1][:n]
    top_courses = df.loc[top_indices].sort_values('relevance', ascending=False)

    return top_courses

In [ ]:
df_task_43 = search_course_43(prompt, n)
df_task_43

,id,title,qualification,raw_text,author_entities,relevance,preprocessed_text
3864,15649,Установки класса Mega-Science в России и за ру...,master,Установки класса Mega-Science в России и за ру...,"[Принцип работы лазера, Линейные ускорители, К...",17,установка класс megascience россия рубеж устан...
3822,15563,Установка класса Mega-Science в России и за ру...,master,Установка класса Mega-Science в России и за ру...,"[Принцип работы лазера, Линейные ускорители, К...",17,установка класс megascience россия рубеж устан...
3499,14946,Подготовительный курс по физике,bachelor,Подготовительный курс по физике. Зачем на вело...,"[Электростатика, Электромагнитные волны, Элект...",16,подготовительный курс физика велосипед различн...
4017,15921,Подготовительный курс по физике,All_levels,Подготовительный курс по физике. Зачем на вело...,"[Электростатика, Электромагнитные волны, Элект...",16,подготовительный курс физика велосипед различн...
7915,24023,Дополнительные главы физики,bachelor,Дополнительные главы физики. Факультативный ку...,"[Электричество и магнетизм, Термодинамика, Ст...",7,дополнительный глава физика факультативный кур...


# Задание 4.4

Напишите функцию, которая по датасетам из заданий 2 и 3 для любой дисциплины по описанию дисциплины генерирует список учебных сущностей. Протестируйте эту функцию, используя датасет из этого задания.